In [11]:
import json
from nltk.corpus import stopwords

with open('data.json') as f:
    raw_json = json.load(f)

In [30]:
data = []
get_word_tokens = lambda words: [word.lower() for word in words if word.isalpha() and word.lower() not in stopwords.words('english')]
    
for index, i in enumerate(raw_json):
    qst_words = get_word_tokens(i['question'].split(' '))
    ans_words = get_word_tokens(i['answer'].split(' '))
    
    keywords = qst_words + ans_words
    
    data.append({
        'index': index,
        'question': i['question'],
        'answer': i['answer'],
        'keywords': keywords
    })

In [186]:
data

[{'index': 0,
  'question': 'How can different animals perceive different colors ? --T-- ',
  'answer': 'Larger animals ( including humans ) have special cells in the back of their eyes called cone cells . Each type of cone cell reacts to a specific wavelength : that is , light whose color corresponds to a specific part of the rainbow . When the cell is hit by light with the right wavelength , it sends a signal to the brain , which combines information from all of the cone types to perceive the full color range . What colors each animal sees depends on the number of different cone types , and what wavelength they react to . For example , most humans have three cell types which perceive violet , green , and yellow , but some of us lack one of these types , leading to green - red color - blindness . Some species of bird or fish on the other hand have four different cell types , giving them a very different perception of colors .',
  'keywords': ['different',
   'animals',
   'perceive',


In [38]:
# Inverted mapping of keywords to document files
inverted_json = {}

for index, item in enumerate(data):
    for keyword in item['keywords']:
        if keyword in inverted_json:
            inverted_json[keyword]['freq'] += 1
            inverted_json[keyword]['doc'].append(index)
        else:
            inverted_json[keyword] = {
                'freq': 1,
                'doc': [index]
            }

# Remove all duplicate values
for key, val in inverted_json.items():
    val['doc'] = list(set(val['doc']))

In [39]:
inverted_json

{'different': {'freq': 9, 'doc': [0, 4, 5]},
 'animals': {'freq': 4, 'doc': [0, 8, 6]},
 'perceive': {'freq': 3, 'doc': [0]},
 'colors': {'freq': 3, 'doc': [0]},
 'larger': {'freq': 3, 'doc': [0, 6]},
 'including': {'freq': 1, 'doc': [0]},
 'humans': {'freq': 5, 'doc': [0, 8, 7]},
 'special': {'freq': 4, 'doc': [0, 8]},
 'cells': {'freq': 3, 'doc': [0, 8]},
 'back': {'freq': 1, 'doc': [0]},
 'eyes': {'freq': 1, 'doc': [0]},
 'called': {'freq': 3, 'doc': [0, 1, 4]},
 'cone': {'freq': 4, 'doc': [0]},
 'type': {'freq': 1, 'doc': [0]},
 'cell': {'freq': 5, 'doc': [0, 7]},
 'reacts': {'freq': 1, 'doc': [0]},
 'specific': {'freq': 3, 'doc': [0, 4]},
 'wavelength': {'freq': 3, 'doc': [0]},
 'light': {'freq': 2, 'doc': [0]},
 'whose': {'freq': 1, 'doc': [0]},
 'color': {'freq': 4, 'doc': [0, 4]},
 'corresponds': {'freq': 1, 'doc': [0]},
 'part': {'freq': 2, 'doc': [0, 6]},
 'rainbow': {'freq': 1, 'doc': [0]},
 'hit': {'freq': 1, 'doc': [0]},
 'right': {'freq': 2, 'doc': [0, 9]},
 'sends': {'fr

In [43]:
inverted_json_keys = list(inverted_json.keys())

In [177]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.children = {}
        self.end = False
        self.suggestion_list = []
        self.base = ''
        
class Trie:
    def __init__(self):
        self.root = TreeNode('')
        
    def insert(self, word):
        root = self.root
        for index, letter in enumerate(word):
            if letter not in root.children:
                root.children[letter] = TreeNode(letter)
            root = root.children[letter]
        if index == len(word)-1: root.end = True

In [180]:
# All keywords formed in a trie ds
trie_root_node = Trie()

for word in inverted_json_keys:
    trie_root_node.insert(word)
    
prefix_hash_table = {}

# Organising the Tries
def dfs(root, curr_word=''):
    root.base = curr_word
    prefix_hash_table[root.base] = root.suggestion_list
    if root.end: 
        root.suggestion_list.append(curr_word)
        return [curr_word]
    
    for val, node in root.children.items():
        to_return = dfs(node, curr_word+node.val)
        root.suggestion_list += to_return
    
    return root.suggestion_list
        
dfs(trie_root_node.root)

# Remove the null entry
prefix_hash_table.pop('', None)

['different',
 'difference',
 'difficult',
 'directed',
 'disease',
 'digest',
 'depends',
 'death',
 'decades',
 'date',
 'drinking',
 'dried',
 'animal',
 'amplify',
 'amounts',
 'air',
 'art',
 'arbitrary',
 'areas',
 'actually',
 'acquired',
 'access',
 'away',
 'already',
 'also',
 'alive',
 'adults',
 'age',
 'aging',
 'affect',
 'ate',
 'attach',
 'able',
 'perceive',
 'perception',
 'period',
 'part',
 'passed',
 'produce',
 'probably',
 'properly',
 'protein',
 'protected',
 'process',
 'predators',
 'present',
 'prey',
 'piece',
 'places',
 'plant',
 'possible',
 'population',
 'pods',
 'pulp',
 'color',
 'column',
 'cone',
 'contrary',
 'contain',
 'corresponds',
 'combines',
 'composer',
 'compound',
 'compared',
 'cover',
 'coffee',
 'could',
 'cows',
 'cocoa',
 'cell',
 'called',
 'case',
 'came',
 'cause',
 'classified',
 'classical',
 'clarinets',
 'create',
 'creation',
 'change',
 'changing',
 'character',
 'children',
 'chemical',
 'chocolate',
 'large',
 'lack',
 'l

In [181]:
# View All suggestion list
def dfs(root, curr_word=''):
    print(f'root: {root.val}, base: {root.base}, suggestion_list: {root.suggestion_list}')
    for val, node in root.children.items():
        dfs(node, curr_word)
        
dfs(trie_root_node.root)

root: , base: , suggestion_list: ['different', 'difference', 'difficult', 'directed', 'disease', 'digest', 'depends', 'death', 'decades', 'date', 'drinking', 'dried', 'animal', 'amplify', 'amounts', 'air', 'art', 'arbitrary', 'areas', 'actually', 'acquired', 'access', 'away', 'already', 'also', 'alive', 'adults', 'age', 'aging', 'affect', 'ate', 'attach', 'able', 'perceive', 'perception', 'period', 'part', 'passed', 'produce', 'probably', 'properly', 'protein', 'protected', 'process', 'predators', 'present', 'prey', 'piece', 'places', 'plant', 'possible', 'population', 'pods', 'pulp', 'color', 'column', 'cone', 'contrary', 'contain', 'corresponds', 'combines', 'composer', 'compound', 'compared', 'cover', 'coffee', 'could', 'cows', 'cocoa', 'cell', 'called', 'case', 'came', 'cause', 'classified', 'classical', 'clarinets', 'create', 'creation', 'change', 'changing', 'character', 'children', 'chemical', 'chocolate', 'large', 'lack', 'light', 'lips', 'like', 'living', 'leading', 'let', 'lo

In [82]:
# DFS on tries to get the words

q = [trie_root_node_test.root]
# v = [trie_root_node]
curr_word = ''

while q:
    root = q.pop()
    print(root.val)
    curr_word += root.val
    if root.end:
#         root.suggestion_list.append(curr_word)
        print(curr_word)
#         curr_word = curr_word[:-1]
    for val, node in root.children.items():
        q.append(node)


c
o
m
m
o
n
common
l
d
commonld
a
s
t
commonldast
r
commonldastr
m
e
r
o
n
commonldastrmeron
l
commonldastrmeronl


In [168]:
# All keywords formed in a trie
trie_root_node_test = Trie()

test_words = ['camel', 'car', 'cast', 'cameron', 'cold', 'common']

for word in test_words:
    trie_root_node_test.insert(word)
    
prefix_hash_table = {}

# Organising the Tries
def dfs(root, curr_word=''):
    root.base = curr_word
    prefix_hash_table[root.base] = root.suggestion_list
    if root.end: 
        root.suggestion_list.append(curr_word)
        return [curr_word]
    
    for val, node in root.children.items():
        to_return = dfs(node, curr_word+node.val)
        root.suggestion_list += to_return
    
    return root.suggestion_list
        
dfs(trie_root_node_test.root)

# Remove the null entry
prefix_hash_table.pop('', None)

['camel', 'cameron', 'car', 'cast', 'cold', 'common']

In [169]:
# View All suggestion list
def dfs(root, curr_word=''):
    print(f'root: {root.val}, base: {root.base}, suggestion_list: {root.suggestion_list}')
    for val, node in root.children.items():
        dfs(node, curr_word)
        
dfs(trie_root_node_test.root)

root: , base: , suggestion_list: ['camel', 'cameron', 'car', 'cast', 'cold', 'common']
root: c, base: c, suggestion_list: ['camel', 'cameron', 'car', 'cast', 'cold', 'common']
root: a, base: ca, suggestion_list: ['camel', 'cameron', 'car', 'cast']
root: m, base: cam, suggestion_list: ['camel', 'cameron']
root: e, base: came, suggestion_list: ['camel', 'cameron']
root: l, base: camel, suggestion_list: ['camel']
root: r, base: camer, suggestion_list: ['cameron']
root: o, base: camero, suggestion_list: ['cameron']
root: n, base: cameron, suggestion_list: ['cameron']
root: r, base: car, suggestion_list: ['car']
root: s, base: cas, suggestion_list: ['cast']
root: t, base: cast, suggestion_list: ['cast']
root: o, base: co, suggestion_list: ['cold', 'common']
root: l, base: col, suggestion_list: ['cold']
root: d, base: cold, suggestion_list: ['cold']
root: m, base: com, suggestion_list: ['common']
root: m, base: comm, suggestion_list: ['common']
root: o, base: commo, suggestion_list: ['common

In [183]:
inverted_json

{'different': {'freq': 9, 'doc': [0, 4, 5]},
 'animals': {'freq': 4, 'doc': [0, 8, 6]},
 'perceive': {'freq': 3, 'doc': [0]},
 'colors': {'freq': 3, 'doc': [0]},
 'larger': {'freq': 3, 'doc': [0, 6]},
 'including': {'freq': 1, 'doc': [0]},
 'humans': {'freq': 5, 'doc': [0, 8, 7]},
 'special': {'freq': 4, 'doc': [0, 8]},
 'cells': {'freq': 3, 'doc': [0, 8]},
 'back': {'freq': 1, 'doc': [0]},
 'eyes': {'freq': 1, 'doc': [0]},
 'called': {'freq': 3, 'doc': [0, 1, 4]},
 'cone': {'freq': 4, 'doc': [0]},
 'type': {'freq': 1, 'doc': [0]},
 'cell': {'freq': 5, 'doc': [0, 7]},
 'reacts': {'freq': 1, 'doc': [0]},
 'specific': {'freq': 3, 'doc': [0, 4]},
 'wavelength': {'freq': 3, 'doc': [0]},
 'light': {'freq': 2, 'doc': [0]},
 'whose': {'freq': 1, 'doc': [0]},
 'color': {'freq': 4, 'doc': [0, 4]},
 'corresponds': {'freq': 1, 'doc': [0]},
 'part': {'freq': 2, 'doc': [0, 6]},
 'rainbow': {'freq': 1, 'doc': [0]},
 'hit': {'freq': 1, 'doc': [0]},
 'right': {'freq': 2, 'doc': [0, 9]},
 'sends': {'fr

In [170]:
prefix_hash_table

{'c': ['camel', 'cameron', 'car', 'cast', 'cold', 'common'],
 'ca': ['camel', 'cameron', 'car', 'cast'],
 'cam': ['camel', 'cameron'],
 'came': ['camel', 'cameron'],
 'camel': ['camel'],
 'camer': ['cameron'],
 'camero': ['cameron'],
 'cameron': ['cameron'],
 'car': ['car'],
 'cas': ['cast'],
 'cast': ['cast'],
 'co': ['cold', 'common'],
 'col': ['cold'],
 'cold': ['cold'],
 'com': ['common'],
 'comm': ['common'],
 'commo': ['common'],
 'common': ['common']}

In [182]:
with open('sample_indexing.js', 'w') as w:
    w.write('data = ')
    json.dump(prefix_hash_table, w)

In [184]:
with open('inverted_indexing.js', 'w') as w:
    w.write('invertedIndex = ')
    json.dump(inverted_json, w)

In [185]:
with open('documents.js', 'w') as w:
    w.write('docIndex = ')
    json.dump(data, w)